# Chapter 3
## Loops and Iterators

# Enumerate
It's better than len + range!

In [ ]:
# Enumerate makes range(len(x)) pointless

for i, thing in enumerate(["a", "b", "c"]):
    print(i, thing)

In [ ]:
# Enumerate works by creating a generator that yields the enumerated item with an integer index

enumerated = enumerate(["a", "b", "c"])
next(enumerated)

In [ ]:
for i, thing in enumerated:  # Discuss
    print(i, thing)

In [ ]:
# What will this do?
next(enumerated)

Aside: generator; what the flip?! DISCUSS

## Make use of `zip`

In [ ]:
from faker import Faker

fake = Faker()

# DID YOU KNOW? List comprehensions edition
# https://wiki.haskell.org/List_comprehension

some_names = [fake.name() for _ in range(9)]
char_counts = [len(n) for n in some_names]

Cool cool. But I want to access data from BOTH lists at the same time, now what?!

In [ ]:
longest_name = None
biggest_count = 0

for i in range(len(some_names)):
    count = char_counts[i]
    if count > biggest_count:
        longest_name = some_names[i]
        biggest_count = count

print(longest_name)

Hideous!!!

Surely, there's a better way Mr. Slatkin?!

In [ ]:
# Generators again?!
longest_name = None
biggest_count = 0

for name, count in zip(some_names, char_counts):
    if count > biggest_count:
        longest_name = name
        biggest_count = count

print(longest_name)

Wow! Thanks Mr. Slatkin! I'm sure there's no caveats right? Like if the lists aren't the same size?

In [ ]:
some_names.append("Video Center Rulez")
for name, count in zip(some_names, char_counts):
    print(name, count)

Damn! So `zip` will only yield a tuple until one of the wrapped sequences is exhausted. Well, good thing there's `strict` if I really care about truncation (or the cool itertools lib)

In [ ]:
try:
    for name, count in zip(some_names, char_counts, strict=True):
        print(name, count)
except ValueError:
    print("No way man!")

In [ ]:
from itertools import zip_longest

for name, count in zip_longest(some_names, char_counts, fillvalue=10000):
    print(name, count)

### Things to Remember
- zip can be used to iterate over multiple iterators in parallel
- zip makes a lazy generator that pops out tuples
- zip (by default) silently truncates output to the length of the shorter iterator
    - use strict to avoid this

# Avoid Else Blocks After for and while Loops

Yes, this _is_ a thing you can do in Python and _yes_ it _is_ confusing.

In [ ]:
for name in some_names[:2]:
    print(name)
else:
    print("Done!")

Why did the else execute? Shouldn't it have... done nothing? The list wasn't empty!!! Why doesn't this work like try/except/else?!

In [ ]:
for x in []:
    print("Unreachable code baby!")
else:
    print("And yet here we are.")

In theory, the `else` is sort of meant to be combined with `break`, in a search operation

In [ ]:
for name in some_names:
    if name == "Video Center Rulez":
        print(f"We found {name}!")
        break
else:
    print("No results found!")

Still, kind of confusing no? Could instead just use a result variable:

In [ ]:
def find_name(names, target) -> bool:
    found_it = False
    for name in names:
        if name == target:
            found_it = True
            break

    return found_it


find_name(some_names, "Video Center Rulez")

### Things to Remember
- the else block on for is goofy, probably just avoid using it

# Never Use for Loop Variables After the Loop Ends
Just because the loop ends, doesn't mean its var goes out of scope...

In [ ]:
for i in range(10):
    print(i)
print(i)

It's tempting to use this in _clever_ ways, but really, just avoid it, because you can't always be sure it will be there:

In [ ]:
for x in []:
    print(x)
# print(x)

In [ ]:
# But it gets worse!

for i in []:
    print(i)
print(i)

Whoops! That's kinda confusing! Yeah just.. don't do this. This leakage behavior is an artifact of how Python builds it's syntax tree. Note how this behavior does _not_ happen on list and generator comprehensions.

# Be Defensive when Iterating over Arguments

Remember the `StopIteration` thing I showed earlier?

In [ ]:
def my_number_gen():
    for i in range(4):
        yield i


gen1 = my_number_gen()
gen2 = my_number_gen()

In [ ]:
for n in gen1:
    print(n)

In [ ]:
while True:
    print(next(gen2))

DISCUSS: why it do that tho?

Python `for` loops automatically except (to halt) on a `StopIteration` exception; as does the `list` constructor!

## Defend yourself

In [ ]:
# Explicitly exhaust the generator and kep its contents in a list:

list_of_stuff = list(my_number_gen())
list_of_stuff

In [ ]:
# But that approach ain't great: what if the generator has 400,000,000 strings?

# Let's just make a container ourselves!
class MyIter:
    def __iter__(self):
        for i in range(4):
            yield i


it1 = MyIter()
next(iter(it1))

In [ ]:
for x in iter(it1):
    print(x)

In [ ]:
# Okay?
def do_it(nums):
    for x in nums:
        print(x)
    return sum(nums)


gen3 = iter(it1)

do_it(gen3)

Whoops! Not quite what we expected right?

In [ ]:
from collections.abc import Iterator


def do_it_defensively(nums):
    if isinstance(nums, Iterator):
        raise TypeError("Give me a container nerd!")
    for x in nums:
        print(x)
    return sum(nums)


do_it_defensively(gen3)

In [ ]:
do_it_defensively(list(my_number_gen()))

# Never Modify Containers While Iterating over Them; Use Copies or Caches Instead
Not just Python advice!

In [ ]:
my_dict = {"red": 1, "blue": 2}

for key in my_dict:
    if key == "blue":
        my_dict["yellow"] = 3

In [ ]:
# but it lets you do this:
for key in my_dict:
    if key == "blue":
        my_dict["blue"] = 3
my_dict

Index overwrites also work fine on `set` and `list` types!

Speaking of lists and indexes: do NOT insert an element before the current iterator position.

In [ ]:
my_list = [1, 2, 3]
for num in my_list:
    print(num)
    if num == 2:
        my_list.insert(0, 4)

Given the inconsistency, make a copy instead of doing modification-in-place.

In [ ]:
my_list = [1, 2, 3]
my_dict = {"red": 1, "blue": 2}

for x in list(my_list):
    print(x)
    if x == 2:
        my_list.insert(0, 4)
print(my_list)

for k in list(my_dict.keys()):
    if k == "blue":
        my_dict["green"] = 4
print(my_dict)

In [ ]:
# For huge datasets, stage modifications:
my_pretend_huge_dict = {"red": 1, "blue": 2}
staged = {}

for k in my_pretend_huge_dict:
    if k == "blue":
        staged["green"] = 4
my_pretend_huge_dict.update(staged)
print(my_pretend_huge_dict)

In [ ]:
# Once more, but with checking the vals too
my_pretend_huge_dict = {"red": 1, "blue": 2, "green": 3}
staged = {}

for k in my_pretend_huge_dict:
    if k == "blue":
        staged["green"] = 4
    val = my_pretend_huge_dict[k]
    other_val = staged.get(k)
    if val == 4 or other_val == 4:
        staged["yellow"] = 5
my_pretend_huge_dict.update(staged)
print(staged)
print(my_pretend_huge_dict)

# Pass Iterators to `any` and `all` for Efficient Short-Circuiting Logic

In [ ]:
# FLIP SOME COINS DAWG
import random


def flippem():
    if random.randint(0, 1) == 0:
        return "heads"
    else:
        return "tails"


def is_it_heads():
    return flippem() == "heads"


flips = [is_it_heads() for _ in range(10)]
flips

This is fine, but does a lot more work than it needs to (imagine this was an expensive operation, and not a simple coin flip).

Instead we can use `all` which will step through an iter and halt early:

In [ ]:
flip_erator = (is_it_heads() for _ in range(10))
type(flip_erator)

In [ ]:
all(flip_erator)

There's also `any` which will tell us if _any_ of the outputs are `True`. Like `all` it halts when it finds what it's looking for. Didn't have to climb highest mountain, nor run through the fields, etc.

In [ ]:
any(flip_erator)

In [ ]:
any([False, False])

Picking between the two is straight-forward:

- I want to end on the first False: use `all`
- I want to end on the first True: use `any`

Consider `itertools` for Working with Iterators and Generators

In [ ]:
import itertools

list(itertools.chain([1, 2, 3], [4, 5, 6]))

In [ ]:
list(itertools.repeat("yo", 5))

In [ ]:
cycle_itr = itertools.cycle([111, 222])
result = [next(cycle_itr) for _ in range(5)]
print(result)

In [ ]:
values = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# handy, so you don't have to keep count of generator values or indexes
first_five = itertools.islice(values, 5)
middle_odds = itertools.islice(values, 2, 8, 2)

print(list(first_five))
print(list(middle_odds))

In [ ]:
less_than_seven = itertools.takewhile(lambda x: x < 7, values)
print(list(less_than_seven))

In [ ]:
seven_and_above = itertools.dropwhile(lambda x: x < 7, values)
print(list(seven_and_above))

In [ ]:
# Batched is pretty cool, no overlaps!

it = itertools.batched([1, 2, 3, 4, 5, 6, 7], 3)
print(list(it))

In [ ]:
# pairwise goes pair by pair, with overlaps

it = itertools.pairwise([1, 2, 3, 4, 5, 6, 7])
print(list(it))

In [ ]:
# Permutations!
print(list(itertools.permutations([1, 2, 3], 3)))

# BONUS: What is the Big O of this?

In [ ]:
# Combinations!
print(list(itertools.combinations([1, 2, 3], 2)))